In [15]:
import pandas as pd
import nltk
from nltk.stem import PorterStemmer as stemmer
from nltk.corpus import stopwords
import re
from tqdm import tqdm

In [16]:
reviews_labeled = "./data/reviews_labeled.csv"
reviews_scored = "./data/reviews_scored.csv"

In [17]:
dataset1 = pd.read_csv(reviews_labeled)
dataset2 = pd.read_csv(reviews_scored)

In [18]:

dataset1.head()

,User_ID,Description,Browser_Used,Device_Used,Is_Response
0,id10326,The room was kind of clean but had a VERY stro...,Edge,Mobile,not happy
1,id10327,I stayed at the Crown Plaza April -- - April -...,Internet Explorer,Mobile,not happy
2,id10328,I booked this hotel through Hotwire at the low...,Mozilla,Tablet,not happy
3,id10329,Stayed here with husband and sons on the way t...,InternetExplorer,Desktop,happy
4,id10330,My girlfriends and I stayed here to celebrate ...,Edge,Tablet,not happy


In [19]:

dataset2.head()

,Review,Rating
0,nice hotel expensive parking got good deal sta...,4
1,ok nothing special charge diamond member hilto...,2
2,nice rooms not 4* experience hotel monaco seat...,3
3,"unique, great stay, wonderful time hotel monac...",5
4,"great stay great stay, went seahawk game aweso...",5


In [20]:
dataset1 = dataset1.rename({"Description": "text", "Is_Response": "sentiment"}, axis=1)[["text", "sentiment"]]
dataset1.replace("not happy", 0, inplace=True)
dataset1.replace("happy", 1, inplace=True)

In [21]:
dataset2 = dataset2.rename({"Review": "text", "Rating": "sentiment"}, axis=1)
dataset2 = dataset2[dataset2["sentiment"] != 3]

In [22]:
sentiments = [0 if x < 3 else 1 for x in dataset2["sentiment"]]

In [23]:
dataset2["sentiment"] = sentiments

In [24]:
dataset = pd.concat([dataset1, dataset2])

In [25]:
# PreProcessing - Removed stopwords + numbers + special characters + converting to lower case
def preProcess(text):
    sents = nltk.sent_tokenize(text)

    for i in range(len(sents)):
        words = nltk.word_tokenize(sents[i])
        for j in range(len(words)):  #word in words:
            if words[j] not in set(stopwords.words('english')): 
                words[j] = re.sub('[^A-Za-z]+','', words[j]) # maybe add numbers
                words[j] = words[j].lower()
                words[j] = stemmer().stem(words[j])
        sents[i] = ' '.join([w for w in words if w != ""])
    
    sents = ' '.join(sents)
    return sents

print(preProcess("This is an :) \n\n 123 test :-( apple!"))

thi is an test appl


In [26]:
t = list(dataset["text"])
print(t[0])

The room was kind of clean but had a VERY strong smell of dogs. Generally below average but ok for a overnight stay if you're not too fussy. Would consider staying again if the price was right. Breakfast was free and just about better than nothing.


In [27]:
dataset.reset_index(drop=True, inplace=True)

In [28]:
texts = list(dataset["text"])
for i in tqdm(range(len(texts))):
    dataset.loc[i, ["text"]] = preProcess(texts[i])
    
dataset.to_csv("./data/cleanedDataset.csv", header=True, index=False)

100%|██████████| 57239/57239 [16:16<00:00, 58.62it/s]  
